In [ ]:
from scipy.spatial import Delaunay
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
from matplotlib import cm
import time
%matplotlib

In [ ]:
"""
create class called grid to collect the grids
create flattened grid of structure (ngrid**3, 3 - xyz)
"""

class grid:
    """
    Generates a grid object with both flattened and mesh versions of cartesian
    and toroidal grid.
    """
    def __init__(self, ngrid_cart, ngrid_tor, R0):
        self.R, self.theta, self.phi = np.meshgrid(np.linspace(1e-2, 1,ngrid_tor), 
                                                   np.linspace(0,2*np.pi, ngrid_tor), 
                                                   np.linspace(0,2*np.pi, 2*ngrid_tor),
                                                   indexing='ij')
        self.cart_x, self.cart_y, self.cart_z = np.meshgrid(np.linspace(-3,3, ngrid_cart),
                                                            np.linspace(-3,3, ngrid_cart),
                                                            np.linspace(-1,1, ngrid_cart),
                                                            indexing='ij')
        self.tor_x = (R0 + self.R*np.cos(self.theta))*np.cos(self.phi)
        self.tor_y = (R0 + self.R*np.cos(self.theta))*np.sin(self.phi)
        self.tor_z = self.R*np.sin(self.theta)
        
        self.cartflat = np.zeros((ngrid_cart**3, 3))
        self.cartflat[:,0] = self.cart_x.flatten()
        self.cartflat[:,1] = self.cart_y.flatten()
        self.cartflat[:,2] = self.cart_z.flatten()
        
        self.torflat = np.zeros((2*ngrid_tor**3, 3))
        self.torflat[:,0] = self.tor_x.flatten()
        self.torflat[:,1] = self.tor_y.flatten()
        self.torflat[:,2] = self.tor_z.flatten()
    def plotgrid(self):
        """
        Args:
         - grid object
        Out:
         - None
        """
        fig_gridscat = plt.figure()
        ax_gridscat = fig_gridscat.add_subplot(projection='3d')
        ax_gridscat.scatter(self.cart_x, self.cart_y, self.cart_z, marker='.')
        ax_gridscat.scatter(self.tor_x, self.tor_y, self.tor_z, marker='x')
        ax_gridscat.set_xlabel("$x[\\mathrm{m}]$")
        ax_gridscat.set_ylabel("$y[\\mathrm{m}]$")
        ax_gridscat.set_zlabel("$z[\\mathrm{m}]$")
    def delaunay_tessalation(self):
        """
        Args:
        - grid object
        Out:
         - tri, Delaunay object, delaunay triangulation of the flattened toroidal coords
         - idx_simplices, ndarray np.int64, array of simplicies in indices of the flattened toroidal coords
         - coord_simplicies, ndarray np.float64, same as idx_simplicies but coords not indices
        """
        tri = Delaunay(self.torflat)
        idx_simplices = tri.simplices
        coord_simplices = self.torflat[idx_simplices]
        return tri, idx_simplices, coord_simplices
        
# grid1 = grid(ngrid_cart = 30, ngrid_tor = 10, R0 = 2)
grid1 = grid(30, 10, 2)

print(grid1.torflat)

In [ ]:
"""
To check that the object function works
"""
# grid1.plotgrid()

In [ ]:
"""
Delaunay triangulation.
"""
tri1, idx_tri1, coord_tri1 = grid1.delaunay_tessalation()

#extract the coordinates of the simplices
print(coord_tri1.shape)

In [ ]:
# fig_simplices = plt.figure()
# ax_simplices = fig_simplices.add_subplot(projection = '3d')
# ax_simplices.scatter(coord_tri1[:,:,0], coord_tri1[:,:,1], coord_tri1[:,:,2])

In [ ]:
"""
To check if the tri object function find_simplex that finds the tetrahedron which a cartesian point is in
we plot the point (2,0,0) which we know is embedded by the grid to see if the correct tetrahedron is found.
"""
eucdist_200 = np.sqrt(np.sum((grid1.cartflat - np.array([2,0,0], dtype = np.float32))**2, axis = 1))
print("eucdist_200.shape", eucdist_200.shape)
idx_200 = np.where(np.isclose(eucdist_200, 0, atol = 1.75e-1) == True )[0]
print("shape of indices atol = 1.75e-1 close to point (2,0,0)", idx_200.shape)
print("the distances to the point are", eucdist_200[idx_200])
print("The point chosen:", grid1.cartflat[idx_200[0]])

idx_simplex_200 = tri1.find_simplex(grid1.cartflat[idx_200[0]])
print("idx_simplex_200", idx_simplex_200)
coord_simplex_200 = coord_tri1[idx_simplex_200]
print("coord_simplex_200", coord_simplex_200)

# demonstrate the neighbour function to find all neighbouring simplices to the one embedding coord (2,0,0)
neighbors_200 = tri1.neighbors[idx_simplex_200]
coords_neighbors200 = coord_tri1
print("coords_neighbors200", coords_neighbors200)
print("coords_neighbors200.shape ", coords_neighbors200.shape)

In [ ]:
from scipy.spatial import ConvexHull
import matplotlib as mpl
import mpl_toolkits.mplot3d as a3

"""
Plot the tetrahedron to confirm that we have found which tetrahedron the point is in.
Note!!! If a point is located in one of the faces of a tetrahedron the point will belong a
degenerate number of tetrahedra. The worst case here is if a point is located at one of the vertices.
Then the degeneracy will be all tetrahadra with that point as a vertex.
"""

def plot_tetrhedron_and_interiorpnt(coord_simplex, idx_p):
    """
    Plot of a tetrahedron cell and the interior point to visually confirm that
    the interior point is actually found.
    """
    hull_found = ConvexHull(coord_simplex)
    indices_found = hull_found.simplices
    faces_found = coord_simplex[indices_found]
    print(faces_found)
    fig_foundsimplex = plt.figure()
    ax_foundsimplex = fig_foundsimplex.add_subplot(projection='3d')
    ax_foundsimplex.set_xlim(-3,3)
    ax_foundsimplex.set_ylim(-3,3)
    ax_foundsimplex.set_zlim(-1,1)
    ax_foundsimplex.set_xlabel("$x[\\mathrm{m}]$")
    ax_foundsimplex.set_ylabel("$y[\\mathrm{m}]$")
    ax_foundsimplex.set_zlabel("$z[\\mathrm{m}]$")

    for f in faces_found:
        face = a3.art3d.Poly3DCollection([f])
        face.set_color(mpl.colors.rgb2hex(np.random.rand(3)))
        face.set_edgecolor('k')
        face.set_alpha(0.5)
        ax_foundsimplex.add_collection3d(face)

    ax_foundsimplex.scatter(grid1.cartflat[idx_p,0], 
                            grid1.cartflat[idx_p,1], 
                            grid1.cartflat[idx_p,2], marker='x')

# plot_tetrhedron_and_interiorpnt(coord_simplex = coord_simplex_200, 
#                                 idx_p = idx_200[0])

In [ ]:
"""
Find all tetrahedra which a cartesian point belongs to. The triangle object function returns -1
if the point does not belong to any tetrahedra.
Creating a dictionary with key coord, value vertices of the tetrahedron in which the point is interior.

This is a more elegant solution, but it has a major flaw which is a good point to make to understand how the
Delaunay triangulation works for periodic data. whenever you have som generic values of some intersection in curved
regions of the grid and inside the hole of the torus cells will be generated. This leads to values in places there
should not be any. This is clear after doing a scatter plot of the potential. There you see that the torus hole is
filled with values, but there should be none. Therefore this method had to be done more manually by restricting the
are cells are made by a minmax cube and then scan throughout the gridspace.
"""
def make_dict_coord_vertices(grid_obj, tri_obj = None):
    """
    Args:
     - grid_obj, grid object
     - tri_obj, Delaunay object
    Out:
     - dict_p_tetrahedron, dictionary with key coord and value the vertices 
       of the tetrahedron the point is interior to
    """
    dict_p_tetrahedron = {}
    ncartcoord = int(len(grid_obj.cartflat))
    if tri_obj == None:
        tri, idx_tri, coord_tri = grid_obj.delaunay_tessalation()
    else:
        tri = tri_obj
        idx_tri = tri.simplices
        coord_tri = grid_obj.torflat[idx_tri]
    for i in range(ncartcoord):
        p = grid_obj.cartflat[i]
        idx_simplex_embed_p = tri.find_simplex(p)
        if idx_simplex_embed_p != -1:
            dict_p_tetrahedron[tuple(p)] = coord_tri[idx_simplex_embed_p]
    return dict_p_tetrahedron

dict_p_tetrahedron1 = make_dict_coord_vertices(grid_obj = grid1)

In [ ]:
print("The number of coordinates that is interior to a tetrahedron:", len(dict_p_tetrahedron1))

In [ ]:
"""
Test: We want to plot a random point and tetrahedron of which the point is embedded by.
"""
def test_plot_rnd_pnt_tetra(grid_obj, dict_p_tetra):
    """
    Args:
     - grid_obj, grid object
     - dict_p_tetra, dict, key coord, value coord tetrahedron of which p is interior
    Out:
    - coord_rand, array np.float64, coords of a random point found to be interior
    - plots the point and the tetrahedron cell if it is found interoir to on of the cells
    """
    idx_rand = np.random.randint(low=0, high=grid_obj.cartflat.shape[0]-1, size = 1)
    coord_rand = grid1.cartflat[idx_rand,:][0]
    print(coord_rand)
    try:
        plot_tetrhedron_and_interiorpnt(coord_simplex = dict_p_tetra[tuple(coord_rand)], idx_p = idx_rand)
    except KeyError:
        print("The point is not inside any of the tetrahedra")
    return coord_rand

In [ ]:
# run test on random point until a point is found
test_rndpnt = test_plot_rnd_pnt_tetra(grid_obj = grid1, dict_p_tetra = dict_p_tetrahedron1)

In [ ]:
"""
Now that we have found all coordinates and 
the tetrahedra of which they are interior, we want to assign values to each point
interior to each tetrahedra by using an interpolation scheme.

Interpolation scheme 1:

Weighted values by normalized inverse distance to each vertex of the tetrahedron.
By normalized we mean the sum of the inverse distances should equal 1.
"""

def interpolation_p_in_simplex(p, simplex, dict_vals):
    """
    Args:
     - p, array np.float64, (x,y,z) cart coords of p in tetrahedron/simplex, dim (3,)
     - simplex, ndarray np.float64, (x,y,z) cart coord of each vertex of tetrahedron/simplex, dim (4,3)
     - dict_vals - dict key coord of a vertex, value param value, len = number of grid points in toroidal coord
    Out:
     - paramval, np.float64, param value interpolated from param vals at vertices, dim (1,)
    """
    vals = []
    for i in range(len(simplex)):
        vals.append(dict_vals[tuple(simplex[i])])
    vals = np.asarray(vals, np.float64)
    euc_dists = np.sqrt(np.sum((simplex - p)**2, axis = 1)) + 1e-8
    N = 1/(np.sum(1/euc_dists, axis=0))
    paramval = np.sum(N*vals/euc_dists, axis = 0)
    return paramval

In [ ]:
# run a test if the function works on a random point interior to a tetrahedron
print("Coord: ", test_rndpnt)
def test_interpolfunc(test_point, dict_p_tetra):
    test_potvals = {tuple(dict_p_tetra[tuple(test_point)][0]):1, 
                    tuple(dict_p_tetra[tuple(test_point)][1]):2, 
                    tuple(dict_p_tetra[tuple(test_point)][2]):3, 
                    tuple(dict_p_tetra[tuple(test_point)][3]):4}
    print("Coord vertices of tetrahedron with potvals:\n", test_potvals)
    test_interpolationvals = interpolation_p_in_simplex(p = test_point,
                                                        simplex = dict_p_tetra[tuple(test_point)], 
                                                        dict_vals = test_potvals)
    print(test_interpolationvals)
test_interpolfunc(test_point = test_rndpnt, dict_p_tetra = dict_p_tetrahedron1)

In [ ]:
"""
Define some potential that is easy to interpret. Just to have some values to interpolate from.
"""

def testpot(r):
    return np.exp(-r)

tpot_tor = testpot(r = grid1.R)

tpot_cart = testpot(r = np.sqrt((np.sqrt(grid1.cart_x**2 + grid1.cart_y**2) - 2)**2 + grid1.cart_z**2))

In [ ]:
"""
Now define a potential which varies in all directions. The potential varies periodically in theta and phi direction.
"""
def tpot_alldir_cart(grid_obj):
    r = np.sqrt((np.sqrt(grid_obj.cart_x**2 + grid_obj.cart_y**2) - 2)**2 + grid_obj.cart_z**2)
    phi = np.arctan2(grid_obj.cart_y,grid_obj.cart_x)
    theta = np.arctan2(grid_obj.cart_z,r)
    return np.exp(-r)*np.sin(theta)*np.cos(phi)

def tpot_alldir_tor(grid_obj):
    return np.exp(-grid_obj.R)*np.sin(grid_obj.theta)*np.cos(grid_obj.phi)

In [ ]:
testpot_alldir_cart = tpot_alldir_cart(grid_obj = grid1)
testpot_alldir_tor = tpot_alldir_tor(grid_obj = grid1)

In [ ]:
"""
Function to plot a param as a contour plot for inspection.
Param must be on a meshgrid structure.
"""
def plot_paramslice_cart(param, idx_y):
    im = plt.imshow(param[:,idx_y].T)
    plt.colorbar(im)
# plot_paramslice_cart(param = tpot_cart, idx_y = 15)

In [ ]:
def plot_paramslice_tor(grid_obj, param, idx_phi):
    fig = plt.figure()
    ax = fig.add_subplot()
    c = ax.contourf(np.sqrt(grid_obj.tor_x[:,:,idx_phi]**2 - grid_obj.tor_y[:,:,idx_phi]**2) - 2, 
                    grid_obj.tor_z[:,:,idx_phi], param[:,:,idx_phi])
    fig.colorbar(c)
# plot_paramslice_tor(grid_obj = grid1, param = tpot_tor, idx_phi = 0)

In [ ]:
# plot_paramslice_cart(param = testpot_alldir_cart, idx_y = 10)

In [ ]:
# plot_paramslice_tor(grid_obj = grid1, param = testpot_alldir_tor, idx_phi = 0)

In [ ]:
"""
Now test the interpolation method wiht actual potential data from the exponentially decaying potential.
"""
def make_param_mesh(dict_p_tetra, grid_obj, tpot_torgrid):
    """
    Make a dictionary with key = coord xyz and value = potential value 
    of the potential values connected to grid points from the toroidal grid.
    
    Args:
     - dict_p_tetra, dict, key coord, val coord simplices tetrahedron
      - grid_obj, grid object
      - tpot_torgrid, ndarray np.float64, meshgrid of the parameter values in reactor geometry
    Out:
     - potmesh, ndarray np.float64, meshgrid of the parameter of interest (in this case the el potential)
     - dict_coord_pot_interpol, dict key coord, value interpolation value of the param of interest

    """
    dict_coord_tpot = {}
    for i in range(len(grid_obj.torflat)):
        dict_coord_tpot[tuple(grid_obj.torflat[i])] = tpot_torgrid.flatten()[i]

    # Points interior to one of the tetrahedra that the toroidal volume is divided in
    ps_interior = np.asarray(list(dict_p_tetra.keys()), dtype = np.float64)

    # Make a dictionary with key coord, value interpolated parameter value
    dict_coord_pot_interpol = {}
    
    # Make a meshgrid of interpolated parameter values
    potmesh = np.nan*np.ones(grid_obj.cart_x.shape)
    
    for i in range(len(ps_interior)):
        coord_idx = np.argwhere((grid_obj.cart_x == ps_interior[i][0]) & 
                                (grid_obj.cart_y == ps_interior[i][1]) & 
                                (grid_obj.cart_z == ps_interior[i][2]))[0] 
        potmesh[coord_idx[0], coord_idx[1], coord_idx[2]] = interpolation_p_in_simplex(p = ps_interior[i],
                                                            simplex = dict_p_tetra[tuple(ps_interior[i])], 
                                                            dict_vals = dict_coord_tpot)
        dict_coord_pot_interpol[tuple(ps_interior[i])] = potmesh[coord_idx[0], coord_idx[1], coord_idx[2]]

    return potmesh, dict_coord_pot_interpol

In [ ]:
potmesh1, dict_coord_pot_interpol1 = make_param_mesh(dict_p_tetra = dict_p_tetrahedron1, 
                                                     grid_obj = grid1, 
                                                     tpot_torgrid = tpot_tor)

In [ ]:
potmesh1.shape
# np.max(np.abs(potmesh1-tpot_cart))

In [ ]:
"""
Test the interpolation on one point to and compare with the direct solution on the cartesian grid.
"""
idx_draw = [27,15,15]

print("From direct solution of potential on cartesian grid the coords chosen with potential value are: \n")
print("x:", grid1.cart_x[idx_draw[0], idx_draw[1], idx_draw[2]])
print("y: ", grid1.cart_y[idx_draw[0], idx_draw[1], idx_draw[2]])
print("z: ", grid1.cart_z[idx_draw[0], idx_draw[1], idx_draw[2]])
print("p:", tpot_cart[idx_draw[0], idx_draw[1], idx_draw[2]])

print("\nFrom interpolated solution of potential on cartesian grid the coords chosen with potential value are: \n")
print("xyz", grid1.cartflat[idx_draw[2] + idx_draw[1]*grid1.cart_x.shape[1] + idx_draw[0]*grid1.cart_x.shape[0]**2])
print("p", dict_coord_pot_interpol1[tuple(grid1.cartflat[idx_draw[2] + idx_draw[1]*grid1.cart_x.shape[1] + idx_draw[0]*grid1.cart_x.shape[0]**2])])

In [ ]:
"""
Restructure interpolated potential data in mesh format to plot the potential in a contourplot to
be able to visually compare the direct solution on the cartesian grid to the interpolated values.

Current issue:
I have to restructure the interpolated parameter(potential) values in a mesh such that it is plotable.
"""


fig_testpot_interpol = plt.figure()
ax_testpot_interpol = fig_testpot_interpol.add_subplot(projection='3d')
# plot difference in analytical and interpolated solution
# potscat = ax_testpot_interpol.scatter(grid1.cartflat[:,0], 
#                                       grid1.cartflat[:,1], 
#                                       grid1.cartflat[:,2], 
#                                       c = potmesh1 - tpot_cart, cmap = cm.jet)
#plot just the interpolated solution
potscat = ax_testpot_interpol.scatter(grid1.cartflat[:,0], 
                                      grid1.cartflat[:,1], 
                                      grid1.cartflat[:,2], 
                                      c = potmesh1, cmap = cm.jet)

ax_testpot_interpol.set_xlabel("$x[m]$")
ax_testpot_interpol.set_ylabel("$y[m]$")
ax_testpot_interpol.set_zlabel("$z[m]$")
fig_testpot_interpol.colorbar(potscat)

In [ ]:
"""
Use MMS to confirm that error of grid converges as you increase the number of gridpoints.
Strategy:
1. Establish all grids with increasing number of points in the cartesian grid.
2. Calculate the interpolation
3. Compare potential values with the direct solution on the cartesian grid and compute a 1D measure of the error.
4. Plot error in loglog and see if the error converges.

"""
tik = time.time()

gridlist = []
tpot_cartlist = []
tpot_torlist = []
for i in range(4):
    gridlist.append( grid(ngrid_cart = 10, ngrid_tor = 10*(i+1), R0 = 2) )
    tpot_cartlist.append( testpot(r = np.sqrt((np.sqrt(gridlist[i].cart_x**2 + gridlist[i].cart_y**2) - 2)**2 + 
                                              gridlist[i].cart_z**2)) )
    tpot_torlist.append( testpot(r = gridlist[i].R) )

tok = time.time()
print("This cell use ", tok - tik, "seconds to run.")

In [ ]:
"""
Test the delaunay tessalation on a generalized 3d romboid.
Here you should se that a generalized 3d romboid gets split into 6 tetrahedra.
"""
xx_rom3d, yy_rom3d, zz_rom3d = np.meshgrid(np.array([0,1,0,1,0,1,0,1]), 
                                            np.array([0,0,1,1,0,0,1,1]), 
                                            np.array([0,0,0,0,1,1,1,1]))
rom3d_gridflat = np.zeros((grid1.cart_x[:2, :2, :2].shape[0]**3,) + (3,))
rom3d_gridflat[:,0] = grid1.tor_x[4:(4+2), :2, :2].flatten()
rom3d_gridflat[:,1] = grid1.tor_y[4:(4+2), :2, :2].flatten()
rom3d_gridflat[:,2] = grid1.tor_z[4:(4+2), :2, :2].flatten()


tri_rom3d = Delaunay(rom3d_gridflat)
# print("The shape of the resulting tessalation, should be (6,4)", tri_rom3d.simplices.shape)
coords_simplices = rom3d_gridflat[tri_rom3d.simplices]
coords_simplices.shape

# # plot the vertices of the cell where we check for an interior point.
# figrom3d = plt.figure()
# axrom3d = figrom3d.add_subplot(projection='3d')
# axrom3d.scatter(coords_simplices[:,:,0], coords_simplices[:,:,1], coords_simplices[:,:,2])


def plot_tetrhedron_and_interiorpnt(coord_simplices):
    fig_foundsimplex = plt.figure()
    ax_foundsimplex = fig_foundsimplex.add_subplot(projection='3d')
    ax_foundsimplex.set_xlim(-3,3)
    ax_foundsimplex.set_ylim(-3,3)
    ax_foundsimplex.set_zlim(-1,1)
    ax_foundsimplex.set_xlabel("$x[\\mathrm{m}]$")
    ax_foundsimplex.set_ylabel("$y[\\mathrm{m}]$")
    ax_foundsimplex.set_zlabel("$z[\\mathrm{m}]$")
    for coord_simplex in coord_simplices:
        hull_found = ConvexHull(coord_simplex)
        indices_found = hull_found.simplices
        faces_found = coord_simplex[indices_found]

        for f in faces_found:
            face = a3.art3d.Poly3DCollection([f])
            face.set_color(mpl.colors.rgb2hex(np.random.rand(3)))
            face.set_edgecolor('k')
            face.set_alpha(0.5)
            ax_foundsimplex.add_collection3d(face)
plot_tetrhedron_and_interiorpnt(coord_simplices = coords_simplices)

In [ ]:
def manual_tessalation(grid_obj):
    """
    The problem is that it is not possible to merge two Delaunay objects or add additional points.
    Possible strategy is to do a search and find out for which of the tetrahedra of each 3d romboid 
    each point is interior to.
    
    Args:
     - grid_obj, grid object
    Out:
     - dict_p_tetrahedron, dict key coord, value coords simplices tetrahedron embedding the key coord
    
    """
    dict_p_tetrahedron = {}
    n = 3
    gridflat = np.zeros((n**3,) + (3,))
    print(grid_obj.tor_x.shape)
    for ir in range(grid_obj.tor_x.shape[0]-n):
        for it in range(grid_obj.tor_x.shape[1]):
            for ip in range(grid_obj.tor_x.shape[2]-n):
#                 print("iteration # ", ir*it*ip)
                if grid_obj.tor_x.shape[1]-n <= it:
                    nlow = it-grid_obj.tor_x.shape[1]
                    nhigh = nlow + n-1
                    idxs_t = np.linspace(nlow,nhigh,n, dtype = np.int32)
                    gridflat[:,0] = grid_obj.tor_x[ir:(ir + n), idxs_t, ip:(ip + n)].flatten()
                    gridflat[:,1] = grid_obj.tor_y[ir:(ir + n), idxs_t, ip:(ip + n)].flatten()
                    gridflat[:,2] = grid_obj.tor_z[ir:(ir + n), idxs_t, ip:(ip + n)].flatten()
                else:
                    gridflat[:,0] = grid_obj.tor_x[ir:(ir + n), it:(it + n), ip:(ip + n)].flatten()
                    gridflat[:,1] = grid_obj.tor_y[ir:(ir + n), it:(it + n), ip:(ip + n)].flatten()
                    gridflat[:,2] = grid_obj.tor_z[ir:(ir + n), it:(it + n), ip:(ip + n)].flatten()
                tri = Delaunay(gridflat)
                idx_tri = tri.simplices
                coord_tri = gridflat[idx_tri]

                minx = np.min(gridflat[:,0])
                maxx = np.max(gridflat[:,0])
                miny = np.min(gridflat[:,1])
                maxy = np.max(gridflat[:,1])
                minz = np.min(gridflat[:,2])
                maxz = np.max(gridflat[:,2])
#                     print("minx", minx, "maxx", maxx, "miny", miny, "maxy", maxy, "minz", minz, "maxz", maxz)

                epsx = grid_obj.cart_x[1,0,0] - grid_obj.cart_x[0,0,0]
                epsy = grid_obj.cart_y[0,1,0] - grid_obj.cart_y[0,0,0]
                epsz = grid_obj.cart_z[0,0,1] - grid_obj.cart_z[0,0,0]
            
                idx_inside = np.argwhere((minx - epsx < grid_obj.cartflat[:,0]) & (maxx + epsx > grid_obj.cartflat[:,0]) & 
                             (miny - epsy < grid_obj.cartflat[:,1]) & (maxy + epsy > grid_obj.cartflat[:,1]) & 
                             (minz - epsz < grid_obj.cartflat[:,2]) & (maxz + epsz > grid_obj.cartflat[:,2]) )
                for i in idx_inside:
                    p = grid_obj.cartflat[i][0]
                    idx_simplex_embed_p = tri.find_simplex(p)
                    if idx_simplex_embed_p != -1:
                        p_tuple = tuple(list(p))
                        dict_p_tetrahedron[p_tuple] = coord_tri[idx_simplex_embed_p]                

    return dict_p_tetrahedron

In [ ]:
tik = time.time()
dict_p_tetrahedron_mantest2 = manual_tessalation(grid_obj = gridlist[1])
len(dict_p_tetrahedron_mantest2)
tok = time.time()
print("The algortihm used", tok-tik, "seconds.")

In [ ]:
tik = time.time()
potmesh_mantest2, dict_coord_pot_interpol_mantest2 = make_param_mesh(dict_p_tetra = dict_p_tetrahedron_mantest2, 
                                                     grid_obj = gridlist[1], 
                                                     tpot_torgrid = tpot_torlist[1])
tok = time.time()
print("The algortihm used", tok-tik, "seconds.")

In [ ]:
"""
3D plot of the difference between the analytical and the interpolated potential.
"""

fig_testpot_interpol = plt.figure()
ax_testpot_interpol = fig_testpot_interpol.add_subplot(projection='3d')

potscat = ax_testpot_interpol.scatter(gridlist[1].cart_x, 
                                      gridlist[1].cart_y, 
                                      gridlist[1].cart_z, 
                                      c = potmesh_mantest2 - tpot_cartlist[1], cmap = cm.jet)

ax_testpot_interpol.set_xlabel("$x[m]$")
ax_testpot_interpol.set_ylabel("$y[m]$")
ax_testpot_interpol.set_zlabel("$z[m]$")
fig_testpot_interpol.colorbar(potscat)

In [ ]:
"""
Make a list of meshgrids of the interpolated potential with different cartesian grid resolution.

Takes about 4-5 min to run.
"""
potmesh_tri_list = []
runtimelist = []
for i in range(4):
    tik = time.time()
    dictpintetra = manual_tessalation(grid_obj = gridlist[i])
    parammesh, dict_param = make_param_mesh(dict_p_tetra = dictpintetra,
                                            grid_obj = gridlist[i],
                                            tpot_torgrid = tpot_torlist[i])
    potmesh_tri_list.append( parammesh )
    tok = time.time()
    runtimelist.append(tok-tik)

In [ ]:
potmesh_tri_list[1].shape
# type(potmesh_tri_list[1])

In [ ]:
# plot a slice of the potential
im = plt.imshow(potmesh_tri_list[2][:,5].T)
plt.colorbar(im)

In [ ]:
"""
Now want to find a cell using vtk and mayavi.
"""

from tvtk.api import tvtk
from mayavi.scripts import mayavi2
from PyQt5 import QtCore
import sip
import vtk

In [ ]:
"""
Example vtk.
cellid is the cell reference where find_cell() starts its search

tol2 is the tolerance squared in terms of proximite to a nearby cell/vertex measured in distance?

subid set to default value

pcoord is parametric coordinates of the point, follow counter clockwise with values ranging from (0,0,0) to (1,1,1)
 - every coordinate will then have a value relative to the first vertex of the cell, the value is between 0 and 1

weights set to default, !!! need to find out how they are computed, sums to 1
- probably use the pcoord to calculate dist to each vertex and use this to calc the weight

cid is the index of the gridcell from the flattened gridcell array with dims (n_i -1, n_j -1, n_k - 1)

The parameter to tune here is the tolerance. If you have edgcases where the point is in the plan
between in the boundary between two cells the definition of interoir point is ambigous because the point
could be far away from the vertcies of the boundary plane. The worst case is if it is in centroid of
the boundary plane. So this is the value you need to take into account when setting the tolerance.
"""
# default values
cell = None
cellid = -1
tol2 = 1e-6
subid = vtk.reference(-1)
pcoord = np.zeros((3))
weights = np.zeros(8)
"""
A structured grid object must be made form the tvtk library which is a python wrapper for vtk.
The parent of structured grid (sg) is the vtk dataset object, so the sg object inherits most of the
functions from vtk.vtkDataSet(). Run command vtk.vtk%object_type%? to look at the documentation by simply
replacing %object_type% with the object you want to read about, for instance StructuredGrid.

The single argument in the init of the sg object is the dimensions of the grid, which is in fortran ordering,
which is the opposite of most python libraries. Thats why we reverse the shape by writing [::-1], which
just reverses the order of the tuple.
"""
sgrid = tvtk.StructuredGrid(dimensions = gridlist[3].tor_x.shape[::-1])
sgrid.points = gridlist[3].torflat
cid = sgrid.find_cell(np.array([2.0,2e-2,2e-2], dtype=np.float64), cell, cellid, tol2, subid, pcoord, weights)
cid

In [ ]:
gridlist[3].tor_x.shape

In [ ]:
"""
Find the indices of the cell vertices.
"""
newgridshape = (gridlist[1].tor_x.shape[0]-1, gridlist[1].tor_x.shape[1]-1, gridlist[1].tor_x.shape[2]-1)
ir, it, ip = np.unravel_index(cid, newgridshape)
print(ir, it, ip)


"""
Plot of the vertices of the cell and the interior point [2.6,2e-2,2e-2].
"""
# fig_cell = plt.figure()
# ax_cell = fig_cell.add_subplot(projection='3d')
# ax_cell.scatter(gridlist[1].tor_x[ir:(ir+2), it:(it+2), ip:(ip+2)], 
#                 gridlist[1].tor_y[ir:(ir+2), it:(it+2), ip:(ip+2)], 
#                 gridlist[1].tor_z[ir:(ir+2), it:(it+2), ip:(ip+2)])
# ax_cell.scatter(2.6,2e-2,2e-2)
# ax_cell.set_xlabel("x")
# ax_cell.set_ylabel("y")
# ax_cell.set_zlabel("z")

"""
Here we print the shape of the resulting cell and restructure the data in a mesh desired components.
"""
gridlist[1].tor_x[ir:(ir+2), it:(it+2), ip:(ip+2)].shape
cell_verts_coord = np.zeros((3,2,2,2))
cell_verts_coord[0] = gridlist[1].tor_x[ir:(ir+2), it:(it+2), ip:(ip+2)]
cell_verts_coord[1] = gridlist[1].tor_y[ir:(ir+2), it:(it+2), ip:(ip+2)]
cell_verts_coord[2] = gridlist[1].tor_z[ir:(ir+2), it:(it+2), ip:(ip+2)]
cell_verts_coord

In [ ]:
def make_parammesh_vtk(grid_obj, sgrid, param_torgrid, ret_nans = False):
    """
    Args:
     - grid_obj, grid object defined in one of the cells below
     - sgrid, tvtk.StructureGrid object
     - param_torgrid, type ndarray, parameter of interest on a toroidal meshgrid structure
     - renans, bool, return NaN vales of points found in a cell or not
    Out:
     - parammesh, ndarray, interpolated parameter values on a 3d cartesian grid
    """
    
    # make a dict to store potential nan values that appear in points found in a cell
    dictnans = {}
    notfound = []
    
    # make the mehsgrid for the parameter we want to interpolate form the original data
    parammesh = np.nan*np.ones(grid_obj.cart_x.shape)
    
    # pass initial/default values to the find_cell function
    cell = None
    cid = int(6e3)
    tol2 = 1e-3
    subid = vtk.reference(-1)
    pcoord = np.zeros((3))
    weights = np.zeros(8)
    
    # iterate through points in the cartesian grid
    for p in grid_obj.cartflat:
        #find the cell p is interior to
        cid = sgrid.find_cell(p, cell, cid, tol2, subid, pcoord, weights)
        # if find_cell returns -1 there is either a numerical error or it did not find the point in that cell
        if cid == -1:
            notfound.append(cid)
            continue
        else:            
            # unravel index of each vertex of the cell found from the toroidal grid
            newgridshape = (grid_obj.tor_x.shape[0]-1, grid_obj.tor_x.shape[1]-1, grid_obj.tor_x.shape[2]-1)
            ir, it, ip = np.unravel_index(cid, newgridshape)

            #find the potential values of each vertex of the cell and interpolate the data using weights
            pvals = param_torgrid[ir:(ir+2), it:(it+2), ip:(ip+2)].flatten() * weights
            paramval = np.nansum(pvals.ravel(), axis = 0)

            # find coordinate indices of the point in the cartesian grid structure with cartesian indicies
            coord_idx = np.argwhere((grid_obj.cart_x == p[0]) & 
                                    (grid_obj.cart_y == p[1]) & 
                                    (grid_obj.cart_z == p[2]))[0] 
            # Assign the cartesian parameter mesh the parameter value
            parammesh[coord_idx[0], coord_idx[1], coord_idx[2]] = paramval
            
    # if user input chosen return nans then return nans 
    if ret_nans:
        # generate different return when including the dictionary containing the nans
        return parammesh, dictnans, notfound
    else:
        return parammesh, notfound

In [ ]:
grid_coarse = grid(ngrid_cart = 20, ngrid_tor = 5, R0 = 2)

In [ ]:
tpot_tor_coarse = testpot(r = grid_coarse.R)

In [ ]:
# defining a coarser structured grid
sgrid_coarse = tvtk.StructuredGrid(dimensions=grid_coarse.tor_x.shape[::-1])
sgrid_coarse.points = grid_coarse.torflat

In [ ]:
tik = time.time()
potmesh_vtk_coarse, notfound_coars = make_parammesh_vtk(grid_obj = grid_coarse, 
                                                        sgrid = sgrid_coarse, 
                                                        param_torgrid = tpot_tor_coarse)
tok = time.time()

In [ ]:
im = plt.imshow(potmesh_vtk_coarse[:,10].T)
plt.colorbar(im)

In [ ]:
tik = time.time()

tpot_cartlist = []
tpot_torlist = []
for i in range(len(gridlist)):
    tpot_cartlist.append( testpot(r = np.sqrt((np.sqrt(gridlist[i].cart_x**2 + gridlist[i].cart_y**2) - 2)**2 + 
                                              gridlist[i].cart_z**2)) )
    tpot_torlist.append( testpot(r = gridlist[i].R) )

tok = time.time()
print("This cell use ", tok - tik, "seconds to run.")

In [ ]:
sgrid_list = []
potmesh_vtk_list = []
runtimelist = []
for i in range(len(gridlist)):
    print("iteration #", i)
    tik = time.time()
    sgrid_list.append(tvtk.StructuredGrid(dimensions=gridlist[i].tor_x.shape[::-1]))
    sgrid_list[i].points = gridlist[i].torflat
    parammesh, notfound = make_parammesh_vtk(grid_obj = gridlist[i], 
                                             sgrid = sgrid_list[i], 
                                             param_torgrid = tpot_torlist[i])
    potmesh_vtk_list.append( parammesh )
    tok = time.time()
    runtimelist.append(tok-tik)

In [ ]:
imesh = 2
im = plt.imshow(potmesh_vtk_list[imesh][:,potmesh_vtk_list[imesh].shape[1]//2].T)
plt.colorbar(im)

In [ ]:
"""
Use the l2norm of the difference between the analytical solution and the interpolated solution
as the measure of error related to the resolution of the grid.
"""

def calc_l2_norm(potmesh, tpot_cart, grid_obj, idir):
    normconst = grid_obj.tor_x.shape[idir]**(3/2)
    return np.sqrt(np.nansum((potmesh - tpot_cart)**2))/normconst

l2s = np.zeros((len(gridlist),3))
for i in range(len(l2s)):
    for j in range(l2s.shape[1]):
        l2s[i,j] = calc_l2_norm(potmesh = potmesh_vtk_list[i],
                                tpot_cart = tpot_cartlist[i],
                                grid_obj = gridlist[i],
                                idir = j)


print(l2s[:3,0])
print(l2s[:3,1])
print(l2s[:3,2])

In [ ]:
# make a list over the grid resolution
ngrid_x = np.asarray([grid.tor_x.shape[0] for grid in gridlist])
ngrid_y = np.asarray([grid.tor_x.shape[1] for grid in gridlist])
ngrid_z = np.asarray([grid.tor_x.shape[2] for grid in gridlist])
print(ngrids)
plt.figure()
plt.plot(1/ngrid_x, l2s[:,0], '--',label = "$n_{grid, \\hat{r}}$")
plt.plot(1/ngrid_y, l2s[:,1], '-.',label = "$n_{grid, \\hat{\\theta}}$")
plt.plot(1/ngrid_z, l2s[:,2], '-',label = "$n_{grid, \\hat{\\phi}}$")
plt.xlabel("$\\frac{1}{n_{grid}}$")
plt.ylabel("$L^2(\\Phi(\\mathbf{r}))$")
plt.xscale("log")
plt.yscale("log")
plt.legend()
plt.show()

In [ ]:
"""
Do exponential fit the gridscaling data compared with the l2 norm.
"""
def expofit(ngx, ngy, ngz, l2list):
    print(ngx.shape)
    
    ax, bx = np.polyfit(np.log(1/ngx), np.log(l2list[:,0]), 1, w=np.sqrt(l2list[:,0]))
    ay, by = np.polyfit(np.log(1/ngy), np.log(l2list[:,1]), 1, w=np.sqrt(l2list[:,1]))
    az, bz = np.polyfit(np.log(1/ngz), np.log(l2list[:,2]), 1, w=np.sqrt(l2list[:,2]))

    nabcis = ngx#np.linspace(1e1, 1e3, int(1e3))

    plt.figure()
    plt.plot(1/ngx, l2list[:,0], '--',label = "$n_{grid, \\hat{r}}$")
    plt.plot(1/ngy, l2list[:,1], '-.',label = "$n_{grid, \\hat{\\theta}}$")
    plt.plot(1/ngz, l2list[:,2], '-',label = "$n_{grid, \\hat{\\phi}}$")
    plt.plot(1/ngx, np.exp(ax*np.log(1/nabcis) + bx), '--',label = "$\\approx n_{grid, \\hat{r}}$")
    plt.plot(1/ngy, np.exp(ay*np.log(1/nabcis) + by), '-.',label = "$\\approx n_{grid, \\hat{\\theta}}$")
    plt.plot(1/ngz, np.exp(az*np.log(1/nabcis) + bz), '-',label = "$\\approx n_{grid, \\hat{\\phi}}$")
    plt.xlabel("$\\frac{1}{n_{grid}}$")
    plt.ylabel("$L^2(\\Phi(\\mathbf{r}))$")
    plt.xscale("log")
    plt.yscale("log")
    plt.legend()
    plt.show()
    
expofit(ngx = ngrid_x, ngy = ngrid_y, ngz = ngrid_z, l2list = l2s)

In [ ]:
"""
Use MMS to confirm that error of grid converges as you increase the number of gridpoints.
Strategy:
1. Establish all grids with increasing number of points in the cartesian grid.
2. Calculate the interpolation
3. Compare potential values with the direct solution on the cartesian grid and compute a 1D measure of the error.
4. Plot error in loglog and see if the error converges.

"""
tik = time.time()

gridlist = []
tpot_cartlist = []
tpot_torlist = []
for i in range(10):
    gridlist.append( grid(ngrid_cart = 20, ngrid_tor = 10*(i+1), R0 = 2) )
    tpot_adir_cart = tpot_alldir_cart(grid_obj = gridlist[i])
    tpot_adir_tor = tpot_alldir_tor(grid_obj = gridlist[i])
    tpot_cartlist.append( tpot_adir_cart )
    tpot_torlist.append( tpot_adir_tor )

sgrid_list = []
potmesh_vtk_list = []
runtimelist = []
for i in range(10):
    tik = time.time()
    sgrid_list.append(tvtk.StructuredGrid(dimensions=gridlist[i].tor_x.shape[::-1]))
    sgrid_list[i].points = gridlist[i].torflat
    parammesh, notfound = make_parammesh_vtk(grid_obj = gridlist[i], 
                                                sgrid = sgrid_list[i], 
                                                param_torgrid = tpot_torlist[i])
    potmesh_vtk_list.append( parammesh )
    tok = time.time()
    runtimelist.append(tok-tik)

In [ ]:
runtimelist
plt.figure()
plt.plot([10*(i+1) for i in range(10)], runtimelist)
plt.yscale('log')
plt.xscale('log')

In [ ]:
potmesh_vtk_list = np.array(potmesh_vtk_list)
potmesh_vtk_list.shape

In [ ]:
ngrid_x = np.asarray([grid.tor_x.shape[0] for grid in gridlist])
ngrid_y = np.asarray([grid.tor_x.shape[1] for grid in gridlist])
ngrid_z = np.asarray([grid.tor_x.shape[2] for grid in gridlist])

l2s_adir = np.zeros((potmesh_vtk_list.shape[0], 3))
for i in range(potmesh_vtk_list.shape[0]):
    for j in range(3):
        l2s_adir[i,j] = calc_l2_norm(potmesh = potmesh_vtk_list[i],
                                     tpot_cart = tpot_cartlist[i],
                                     grid_obj = gridlist[i],
                                     idir = j) 


l2s_adir = np.asarray(l2s_adir)
print(l2s_adir[:,0])
print(l2s_adir[:,1])
print(l2s_adir[:,2])

In [ ]:
im = plt.imshow(potmesh_vtk_list[2][:,10].T)
plt.colorbar(im)

In [ ]:
expofit(ngx = ngrid_x, ngy = ngrid_y, ngz = ngrid_x, l2list = l2s_adir)